# CSE4076
# Image and Video Analytics
# Assignment 6 - Task 3

# Name: Mohamed Riyaz S
# Reg No: 21MIA1138

# **Task 3: Facial Recognition to Check Fraud Cases**
**Objective:**
Identify a suspect by comparing their facial features to a reference image to check for fraud cases, using traditional facial recognition techniques.

**Task Description:**

**Load Images and Video:**

Load the reference image of the suspect and a video showing multiple faces.

**Face Detection:**

Use Haar Cascades to detect faces in both the reference image and the video.

**Feature Matching:**

Extract facial features using edge detection or geometric facial features (eye spacing, nose length, etc.).
Compare the features of the faces in the video with the reference face to check for a match.

**Result:**

Output the frames where a match is found and highlight the detected face in the video.

In [2]:
import os
print(os.path.exists(reference_image_path))


True


In [16]:
# Import necessary libraries
from PIL import Image
import cv2
import numpy as np

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the reference image
reference_image_path = '/content/theif image.png'  # Path to the suspect's reference image
reference_image = cv2.imread(reference_image_path)
if reference_image is None:
    print("Error: Could not load the reference image.")
else:
    print("Reference image loaded successfully.")

# Convert the image to RGB format and then to grayscale
reference_image = cv2.cvtColor(reference_image, cv2.COLOR_BGR2RGB)
gray_reference = cv2.cvtColor(reference_image, cv2.COLOR_RGB2GRAY)

# Detect face in the reference image
reference_faces = face_cascade.detectMultiScale(gray_reference, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
if len(reference_faces) == 0:
    print("No face detected in the reference image.")
else:
    x, y, w, h = reference_faces[0]
    reference_face = gray_reference[y:y+h, x:x+w]
    print("Face detected in the reference image.")

# Load the video containing multiple faces
video_path = '/content/video containing multiple faces.mp4'  # Path to the video file
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
else:
    print("Video loaded successfully.")

# Define ORB with a reasonable number of features for better matching
orb = cv2.ORB_create(nfeatures=500)  # Try reducing feature count if needed

# Extract features from the reference face
keypoints_ref, descriptors_ref = orb.detectAndCompute(reference_face, None)

# Ensure descriptors_ref is valid
if descriptors_ref is None:
    print("No descriptors found in the reference face.")
    cap.release()
    cv2.destroyAllWindows()
    exit()

# Initialize variables for tracking match frames
matched_frames = []

# FLANN parameters
FLANN_INDEX_LSH = 6
index_params = dict(algorithm=FLANN_INDEX_LSH,
                    table_number=6,  # 12
                    key_size=12,     # 20
                    multi_probe_level=1)  # 2
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

# Process each frame in the video
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Exit loop if no more frames

    # Convert frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face_in_frame = gray_frame[y:y+h, x:x+w]

        # Extract features from detected face in the frame
        keypoints_frame, descriptors_frame = orb.detectAndCompute(face_in_frame, None)

        # Ensure descriptors_frame is valid
        if descriptors_frame is None:
            continue  # Skip if no features were detected in this face region

        # Debugging: Print sizes of descriptors
        print(f"Descriptors Reference: {descriptors_ref.shape} - Descriptors Frame: {descriptors_frame.shape}")

        # Match features using FLANN
        if descriptors_ref.shape[0] > 0 and descriptors_frame.shape[0] > 0:
            try:
                matches = flann.knnMatch(descriptors_ref, descriptors_frame, k=2)

                # Apply ratio test to filter matches
                good_matches = []
                for match in matches:
                    if len(match) == 2:  # Ensure there are two matches
                        m, n = match
                        if m.distance < 0.7 * n.distance:  # Adjust ratio as needed
                            good_matches.append(m)

                # If enough good matches are found, assume a match with the suspect
                if len(good_matches) > 10:  # Adjust threshold based on testing
                    print(f"Match found in frame {frame_count}.")
                    matched_frames.append(frame_count)

                    # Draw bounding box around the matched face
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, "Suspect", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    # Display the frame with highlighted suspect
                    cv2.imshow("Matched Frame", frame)
                    cv2.waitKey(100)  # Pause to view each frame briefly
                else:
                    print(f"Not enough good matches in frame {frame_count}.")

            except cv2.error as e:
                print(f"FLANN matching error: {e}")

    frame_count += 1

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()

# Output matched frames
if matched_frames:
    print("Frames with suspected match:", matched_frames)
else:
    print("No match found for the suspect in the video.")


Reference image loaded successfully.
Face detected in the reference image.
Video loaded successfully.
Descriptors Reference: (39, 32) - Descriptors Frame: (1, 32)
FLANN matching error: OpenCV(4.10.0) /io/opencv/modules/flann/src/miniflann.cpp:521: error: (-215:Assertion failed) (size_t)knn <= index_->size() in function 'runKnnSearch_'

Descriptors Reference: (39, 32) - Descriptors Frame: (2, 32)
Not enough good matches in frame 1.
Descriptors Reference: (39, 32) - Descriptors Frame: (2, 32)
Not enough good matches in frame 2.
Descriptors Reference: (39, 32) - Descriptors Frame: (5, 32)
Not enough good matches in frame 4.
Descriptors Reference: (39, 32) - Descriptors Frame: (1, 32)
FLANN matching error: OpenCV(4.10.0) /io/opencv/modules/flann/src/miniflann.cpp:521: error: (-215:Assertion failed) (size_t)knn <= index_->size() in function 'runKnnSearch_'

Descriptors Reference: (39, 32) - Descriptors Frame: (8, 32)
Not enough good matches in frame 5.
Descriptors Reference: (39, 32) - Desc